From: https://colab.research.google.com/drive/1VCwcpd_zJ4zWbxBPIfHV6a0uWGcpthnm?usp=sharing

IWBDA 2022 example - LabOP for Opentrons OT2 execution

In [37]:
!git clone https://github.com/Bioprotocols/labop.git
!pip install ./labop
!pip install opentrons

!cd ..

Cloning into 'labop'...
remote: Enumerating objects: 4342, done.
remote: Counting objects: 100% (2332/2332), done.
remote: Compressing objects: 100% (784/784), done.
remote: Total 4342 (delta 1669), reused 2086 (delta 1525), pack-reused 2010
Receiving objects: 100% (4342/4342), 14.69 MiB | 12.59 MiB/s, done.
Resolving deltas: 100% (2934/2934), done.
Processing ./labop
  Preparing metadata (setup.py) ... done
  Using cached https://github.com/rpgoldman/container-ontology/tarball/main
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/rpgoldman/owlery-client.git (to revision v1.0.0) to /private/var/folders/tj/rjb1ztwd5t3dkwv4swm8h4_h0000gn/T/pip-install-cdxnql5v/owlery-client_51478ba6315a443e8cf347996878276d
  Running command git clone --filter=blob:none --quiet https://github.com/rpgoldman/owlery-client.git /private/var/folders/tj/rjb1ztwd5t3dkwv4swm8h4_h0000gn/T/pip-install-cdxnql5v/owlery-client_51478ba6315a443e8cf347996878276d
  Running command git checkout -q e0ea

In [38]:
import labop
from labop_convert.opentrons.opentrons_specialization import OT2Specialization
from labop.execution_engine import ExecutionEngine
from labop_convert.markdown.markdown_specialization import MarkdownSpecialization
from labop. import run_ot2_sim, make_demo_script

import sbol3
import uml
import tyto # for annotation practices in synbio data-sharing
from IPython.display import Markdown, IFrame, Image

import os

# Set up our working directory for protocol files
out_dir = os.path.join(os.path.abspath(""), "out")
if not os.path.exists(out_dir):
    os.mkdir(out_dir)

ModuleNotFoundError: No module named 'labop.utils'

In [ ]:
# Set up the document
print("Setting up the document")
doc = sbol3.Document()
sbol3.set_namespace("http://bbn.com/synbio") # not too sure what a document namespace is

# Import the primitive libraries
labop.import_library("liquid_handling")
labop.import_library("plate_handling")
labop.import_library("spectrophotometry")
labop.import_library("sample_arrays")

protocol = labop.Protocol("OT2_demo")
protocol.name = "OT2 Demo"
protocol.description = "IWBDA 2022 Example Using Opentrons OT2"
doc.add(protocol)

Setting up the document


In [ ]:
# Inspect the available OT2 hardware modules -- can be accessed in the OT2Specialization.EQUIPMENT lookup table
for equipment, agent in OT2Specialization.EQUIPMENT.items():
    print(equipment, agent)

p20_single_gen2 <Agent http://sbols.org/unspecified_namespace/p20_single_gen2>
p300_single_gen2 <Agent http://sbols.org/unspecified_namespace/p300_single_gen2>
p1000_single_gen2 <Agent http://sbols.org/unspecified_namespace/p1000_single_gen2>
p300_multi_gen2 <Agent http://sbols.org/unspecified_namespace/p300_multi_gen2>
p20_multi_gen2 <Agent http://sbols.org/unspecified_namespace/p20_multi_gen2>
p10_single <Agent http://sbols.org/unspecified_namespace/p10_single>
p10_multi <Agent http://sbols.org/unspecified_namespace/p10_multi>
p50_single <Agent http://sbols.org/unspecified_namespace/p50_single>
p50_multi <Agent http://sbols.org/unspecified_namespace/p50_multi>
p300_single <Agent http://sbols.org/unspecified_namespace/p300_single>
p300_multi <Agent http://sbols.org/unspecified_namespace/p300_multi>
p1000_single <Agent http://sbols.org/unspecified_namespace/p1000_single>
temperature_module <Agent http://sbols.org/unspecified_namespace/temperature_module>
tempdeck <Agent http://sbols.or

In [ ]:
#  Set up OT2 machine

# Choose a pipette and set up the OT2 machine
left_pipette = protocol.primitive_step("ConfigureRobot",
                                       instrument=OT2Specialization.EQUIPMENT['p300_single_gen2'],
                                       mount="left")

# Add a tiprack to Deck 1
# (pipette will automatically configure if it detects a compatible tiprack)
tiprack_spec = labop.ContainerSpec("tiprack",queryString="cont:Opentrons96TipRack300uL")
tiprack = protocol.primitive_step("LoadRackOnInstrument", rack=tiprack_spec, coordinates="1")

# Specify a 96 well plate for samples and add it to Deck 2
plate_spec = labop.ContainerSpec("sample_plate",
                                name="calibration plate", # this name shows up in the human-readable protocol
                                queryString="cont:Corning96WellPlate360uLFlat") # choose a 95 well plate
plate = protocol.primitive_step("EmptyContainer", specification=plate_spec)
load_plate = protocol.primitive_step("LoadRackOnInstrument", rack=plate_spec, coordinates="2")

In [ ]:
# Transfer from one well to another
source_well = protocol.primitive_step("PlateCoordinates", source=plate.output_pin("samples"), coordinates="A1")
dest_well = protocol.primitive_step("PlateCoordinates", source=plate.output_pin("samples"), coordinates="B2")
pip1 = protocol.primitive_step("Transfer",
                               source=source_well.output_pin("samples"),
                               destination=dest_well.output_pin("samples"),
                               amount=sbol3.Measure(100, tyto.OM.microliter)
                               )

In [ ]:
# Execute the protocol
# An OT2 script and a markdown script will be generated in the output directory
filename = os.path.join(out_dir, "ot2_demo_labop")
automated_script = filename + ".py"
agent = sbol3.Agent("ot2_machine", name="OT2 Machine")
ee = ExecutionEngine(specializations=[MarkdownSpecialization(filename+".md")])
execution = ee.execute(protocol, agent, id="test_execution")

Initializing execution test_execution


ERROR:/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/labop/execution_engine_utils.py:Could Not Process http://bbn.com/synbio/test_execution/CallBehaviorExecution7: '{"dims": ["aliquot"], "attrs": {}, "data": [true, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false, false], "coords": {"aliquot": {"dims": ["aliquot"], "attrs": {}, "data": ["A1", "B1", "C1", "D

In [ ]:
print(execution.markdown)

# OT2 Demo

IWBDA 2022 Example Using Opentrons OT2


## Protocol Materials:
* Corning 96 Well Plate 360 uL Flat


## Protocol Steps:
1. Obtain a Corning 96 Well Plate 360 uL Flat to contain `calibration plate`
2. Import data for  into provided Excel file.
---
Timestamp: 2023-01-29 23:11:29.013446


In [ ]:
# Print operator instructions
Markdown(ee.specializations[0].markdown)

AttributeError: 'MarkdownSpecialization' object has no attribute 'markdown'

In [ ]:
# Print OT2 script
print(ee.specializations[0].script)

AttributeError: 'MarkdownSpecialization' object has no attribute 'script'